# Student Information

Wangyang He


UIN 625004872


Nickname: heswaggy

Project submission 8

In [1]:
import os
import numpy as np
import cv2
import json
import keras
from keras.models import load_model
from tqdm.notebook import tqdm

In [2]:
!pip install --upgrade youtube_dl

     |████████████████████████████████| 1.9MB 7.4MB/s 


In [3]:
test_videos = '/content/applyvideos'
os.mkdir(test_videos)

In [5]:
import datetime
import youtube_dl, subprocess

#these are the videos link to test, uncomment to download them
urls = [
        'https://www.youtube.com/watch?v=0v7Xlfa3twQ',
        'https://www.youtube.com/watch?v=9WnikBwAl_I',
        'https://www.youtube.com/watch?v=6aPs5cbPhQE',
        'https://www.youtube.com/watch?v=ieZE7z-0Pqw',
        'https://www.youtube.com/watch?v=kRJLUss-j3M',
        'https://www.youtube.com/watch?v=P0_kOnzbPqo',
        'https://www.youtube.com/watch?v=TKad2bvgmaA',
        'https://www.youtube.com/watch?v=Ti9n0GWocGA',
        'https://www.youtube.com/watch?v=muBQhnLYoss',
        'https://www.youtube.com/watch?v=BGNwDYw4I-Y',
        'https://www.youtube.com/watch?v=W7rDhoCopJg',
        'https://www.youtube.com/watch?v=oA6Ijr-wBvk',
        'https://www.youtube.com/watch?v=IN_F50hYSg4',
        'https://www.youtube.com/watch?v=6Whibi7bfdw',
        'https://www.youtube.com/watch?v=998jG2ZZZUo',
        'https://www.youtube.com/watch?v=9xrTKKbekpo',
        'https://www.youtube.com/watch?v=CAtIHCWuebU',
        'https://www.youtube.com/watch?v=F4B6FGw0E4o',
        'https://www.youtube.com/watch?v=SKsOFQYlRmU',
        'https://www.youtube.com/watch?v=qAxtUoSALYA',
        'https://www.youtube.com/watch?v=K7XiEU8yivM',
        'https://www.youtube.com/watch?v=O4xYK0K4SIU',
        'https://www.youtube.com/watch?v=mJ10n8F1_TI',
        'https://www.youtube.com/watch?v=cFCXp_pOPiM',
        'https://www.youtube.com/watch?v=4GrCQujMrwY',
        'https://www.youtube.com/watch?v=i5pKumDDgPo'
         ]

for i, id in enumerate(urls):
  #save videos to the folder we created above
  true = os.path.join(test_videos,"video.{}.mp4".format(i))
  #download videos
  with youtube_dl.YoutubeDL({'format': 'best'}) as ydl:
    try:
      info = ydl.extract_info(id, download=False)
      video = info['entries'][0] if 'entries' in info else info
    except:
      continue
  #parameters
  url = video['url']
  starttime = datetime.timedelta(seconds=0)
  clip_length = datetime.timedelta(seconds=9999)
  subprocess.call(['ffmpeg', '-i', url, '-ss', str(starttime), '-t', str(clip_length), 
                   '-c:v', 'copy', '-c:a', 'copy', true,]) 
  print("finished downloading")

[youtube] 0v7Xlfa3twQ: Downloading webpage
finished downloading
[youtube] 9WnikBwAl_I: Downloading webpage
finished downloading
[youtube] 6aPs5cbPhQE: Downloading webpage
finished downloading
[youtube] ieZE7z-0Pqw: Downloading webpage
finished downloading
[youtube] kRJLUss-j3M: Downloading webpage
[youtube] kRJLUss-j3M: Downloading MPD manifest
finished downloading
[youtube] P0_kOnzbPqo: Downloading webpage
finished downloading
[youtube] TKad2bvgmaA: Downloading webpage
finished downloading
[youtube] Ti9n0GWocGA: Downloading webpage
[youtube] Ti9n0GWocGA: Downloading MPD manifest
finished downloading
[youtube] muBQhnLYoss: Downloading webpage
[youtube] muBQhnLYoss: Downloading MPD manifest
finished downloading
[youtube] BGNwDYw4I-Y: Downloading webpage
finished downloading
[youtube] W7rDhoCopJg: Downloading webpage
finished downloading
[youtube] oA6Ijr-wBvk: Downloading webpage
finished downloading
[youtube] IN_F50hYSg4: Downloading webpage
finished downloading
[youtube] 6Whibi7bfdw: D

In [54]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/Submission 8/cleaningwindows_Xcep_sub8.h5') #this .h5 model file is also avaliable on my github.tamu repo
#loading trained model
print("model loaded")

model loaded


In [89]:
def check_model(img, video, model): #this function will check the video with our model, and predict accruacy to produce JSON
    #get the video itself
    capture = cv2.VideoCapture(img)
    # get the video frame per second
    fps = capture.get(5) 
    # get all frames in the video
    video_frame = int(capture.get(7)) 
    # get video length in seconds
    length = int(video_frame/fps) 
    # get the number of predictions our model will make on the test video
    num_pred = int(length/2) 
    # frame per prediction
    fpp = video_frame // num_pred
    # length per frame
    dis = fpp // 15
    #count the frames for each 2 second clip
    count = 0
    pred_count = 1
    #some parameters to use in the for loop
    pred_time = []
    final_frame = []
    clips = []
    tmp = []

    for i in tqdm(range(video_frame)): #iterating over all the frames of the video
            #using openCV
            ret, frame = capture.read()
            count = count + 1

            if count == dis:
                count = count * 0
                frame=cv2.resize(frame,(150,150),interpolation=cv2.INTER_AREA)
                tmp.append(frame)
                final_frame.append(frame)
                
            if i == ((pred_count * fpp) - 1):
                inpt = np.array(tmp)
                inpt = inpt.astype('float32')
                inpt -= np.mean(inpt)
                inpt /= np.max(inpt)
                tmp = []
                data = np.expand_dims(inpt, axis=0)
                pred = model.predict(data, batch_size = 1) # verbose = 1
                rate = pred[0][1] #prediction score stored
                time_init = (pred_count - 1) / 0.5 #start time
                time_end = pred_count / 0.5  # end time
                #produce JSON file
                if rate > 0.8: #if prediction score better than 80%, add in the JSON file
                    time_result = ['Target: ' +  str(time_init) + 'secs - ' + str(time_end) + 'secs.']
                    json_format = {
                        'videoId': "i5pKumDDgPo",
                        'type': "segment",
                        'startTime': np.float(time_init),
                        'endTime': np.float(time_end),
                        'observer': 'CSCE636Spring2021-heswaggy-8',
                        'isHuman': False,
                        'confirmedBySomeone': False,
                        'rejectedBySomeone': False,
                        'observation': {
                            'label': 'Cleaning windows glass doors etc',
                            'labelConfidence': np.float(rate)
                        }
                    }
                    clips.append(json_format)
                else:
                    time_result = ['Others: ' +  str(time_init) + 'secs - ' + str(time_end) + 'secs.']
                pred_count += 1
                pred_time.append(time_result)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    capture.release()
    cv2.destroyAllWindows()

    with open(video, 'w', encoding='utf-8') as f:
        json.dump(clips, f, indent=4)
    return final_frame

print("finished")

finished


In [9]:
!mkdir ResultOfDetection

In [90]:
final_frame = check_model('/content/applyvideos/video.25.mp4', '/content/ResultOfDetection/ResultOfDetection.25.json', model)

In [91]:
!cp -a '/content/ResultOfDetection/' '/content/drive/MyDrive/Colab Notebooks/Submission 8'